In [1]:
import itertools as it
import os

import biom
from matplotlib import rcParams
import matplotlib.colors as mplc
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy.stats import ttest_rel, ttest_1samp
import seaborn as sn
import statsmodels.formula.api as smf
import statsmodels.api as sms
import skbio

from qiime2 import Artifact, Metadata, MetadataColumn
import qiime2.plugins.diversity.actions as q2_diversity

In [2]:
rcParams['font.sans-serif'] = ['Helvetica', 'Arial']
rcParams['pdf.fonttype'] = 42
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

In [3]:
%matplotlib inline

In [4]:
methods = [
    'reference',
    'otus',
    'asvs',
    'sidle',
    ]

In [5]:
meta = pd.read_csv('data/output/simulation/samples/metadata.tsv', sep='\t', dtype=str)
meta.set_index('sample-id', inplace=True)
meta = meta.loc[meta['set'] == '1']
# meta['color'] = meta['composite'].replace({'infant-1': '#1f78b4', 
#                                            'infant-2': '#a6cee3', 
#                                            'adult-1': '#e31a1c', 
#                                            'adult-2': '#fb9a99'})

In [6]:
meta.shape

(60, 3)

In [7]:
tables = {
    method: {
        artifact: Artifact.load(f'data/output/simulation/merged/{method}/{artifact}.qza')
        for artifact in ['table', 'taxonomy']
    }
    for method in methods
}

In [8]:
def tidy_silva_levels(s):
    for i, level in enumerate(['k', 'p', 'c', 'o', 'f', 'g', 's']):
        s = s.replace(f"D_{i}", level)
    return s

In [9]:
def assemble_taxonomy(method):
    print(method)
    table = tables[method]['table'].view(biom.Table).filter(meta.index)
    taxa = tables[method]['taxonomy'].view(pd.Series)
    taxa = taxa.loc[table.ids(axis='observation')]
    
    if method == 'reference':
        taxa = taxa.apply(lambda x: pd.Series(x.replace("; ", ';').split(';')))
        taxa.replace({'c__Erysipelotrichi': 'c__Erysipelotrichia',
                       'f__Bacteroidales S24-7 group': 'f__S24-7',
                      'f__[Barnesiellaceae]': 'f__Porphyromonadaceae',
                     }, inplace=True)
        taxa.replace({'p__': np.nan, 'c__': np.nan, 'o__': np.nan, 
                      'f__': np.nan, 'g__': np.nan, 's__': np.nan},
                     inplace=True)
        taxa.fillna(method='ffill', axis=1, inplace=True)
    else:
        taxa = taxa.loc[table.ids(axis='observation')].apply(tidy_silva_levels)
        taxa = taxa.apply(lambda x: pd.Series(x.split(';')))
        taxa.fillna('Ambiguous', inplace=True)
        uncultured = {
            t: np.nan for t in np.unique(taxa.values.flatten())
            if (('uncultured' in t) | ('Ambiguous' in t) | ('metagenome' in t) |
                ('unidentified' in t) | ('|' in t))
        }
        taxa.replace(uncultured, inplace=True)
        taxa.replace({'f__Clostridiaceae 1': 'f__Clostridiaceae',
                      }, inplace=True)
        taxa.fillna(method='ffill', axis=1, inplace=True)
        table.add_metadata(taxa.to_dict(orient='index'), axis='observation')
    
    table.add_metadata(taxa.to_dict(orient='index'), axis='observation')
    
    return table

In [10]:
table_with_taxa = {method: assemble_taxonomy(method)
                   for method in methods}

reference
otus
asvs
sidle


# Sequence coverage

## ASVs (Greengenes)

In [11]:
seq_coverage = pd.DataFrame(
    data=[Artifact.load('data/output/simulation/asv_ref/v13-seqs.qza').view(pd.Series), 
          Artifact.load('data/output/simulation/asv_ref/v34-seqs.qza').view(pd.Series), 
          Artifact.load('data/output/simulation/asv_ref/v68-seqs.qza').view(pd.Series),
         ],
    index=['v13', 'v4', 'v68']).notna().T

In [12]:
ref_ids = table_with_taxa['reference'].ids(axis='observation')

In [13]:
ref_coverage = pd.concat(axis=1, objs=[
    seq_coverage.loc[ref_ids], 
    table_with_taxa['reference'].metadata_to_dataframe(axis='observation')
])

In [14]:
ref_coverage[3].unique()

array(['o__Actinomycetales', 'o__Lactobacillales', 'o__Clostridiales',
       'o__Bacteroidales', 'o__Bacillales', 'o__Campylobacterales',
       'o__Thiohalorhabdales', 'o__Enterobacteriales',
       'o__Desulfovibrionales', 'o__Bifidobacteriales',
       'o__Erysipelotrichales', 'o__Chromatiales', 'o__Aeromonadales',
       'o__Anaeroplasmatales', 'o__Verrucomicrobiales',
       'o__Elusimicrobiales', 'o__Turicibacterales', 'o__Pasteurellales',
       'o__Pseudomonadales', 'o__Burkholderiales', 'o__Coriobacteriales',
       'o__Spirochaetales', 'o__RF39', 'o__YS2', 'o__Fusobacteriales',
       'c__Clostridia', 'o__Victivallales', 'o__Neisseriales',
       'c__Gammaproteobacteria', 'o__Streptophyta', 'o__Gemellales',
       'o__Alteromonadales', 'o__Rickettsiales', 'o__RF32',
       'o__Methanobacteriales', 'o__ML615J-28', 'o__[Cerasicoccales]',
       'o__WCHB1-41', 'o__Rhodocyclales', 'o__SHA-98'], dtype=object)

In [15]:
class_coverage = (ref_coverage.groupby(2).sum()[['v13', 'v4', 'v68']] /
                  ref_coverage.groupby(2).count()[['v13', 'v4', 'v68']])
family_coverage = (ref_coverage.groupby(4).sum()[['v13', 'v4', 'v68']] /
                  ref_coverage.groupby(4).count()[['v13', 'v4', 'v68']])

In [16]:
class_coverage

,v13,v4,v68
2,,,
c__4C0d-2,0.636364,1.000000,1.0
c__Actinobacteria,0.218750,1.000000,1.0
c__Alphaproteobacteria,0.666667,1.000000,1.0
c__Bacilli,0.423913,1.000000,1.0
c__Bacteroidia,0.451456,1.000000,1.0
c__Betaproteobacteria,0.421053,1.000000,1.0
c__Chloroplast,1.000000,1.000000,1.0
c__Clostridia,0.551265,0.999334,1.0
c__Coriobacteriia,0.409091,1.000000,1.0


In [17]:
class_coverage.mean(axis=1).round(3)

2
c__4C0d-2                   0.879
c__Actinobacteria           0.740
c__Alphaproteobacteria      0.889
c__Bacilli                  0.808
c__Bacteroidia              0.817
c__Betaproteobacteria       0.807
c__Chloroplast              1.000
c__Clostridia               0.850
c__Coriobacteriia           0.803
c__Deltaproteobacteria      0.889
c__Elusimicrobia            1.000
c__Epsilonproteobacteria    0.714
c__Erysipelotrichia         0.852
c__Fusobacteriia            0.833
c__Gammaproteobacteria      0.790
c__Methanobacteria          0.667
c__Mollicutes               0.762
c__Opitutae                 1.000
c__RF3                      0.889
c__Spirochaetes             0.889
c__Verruco-5                1.000
c__Verrucomicrobiae         0.778
c__[Lentisphaeria]          0.800
dtype: float64

## Aligned Database (Silva)

In [18]:
def load_region(region):
    art = Artifact.load(f'data/output/simulation/database/{region}_map.qza')
    art = art.view(pd.DataFrame)['kmer-length']
    seq_ids = art.index.drop_duplicates()
    return pd.Series(np.array([True] * len(seq_ids)), index=seq_ids, name=region)

db_coverage = pd.concat(axis=1, objs=[
    load_region(region) for region in ['v13', 'v34', 'v68']
]).notna()

In [19]:
db_taxa = Artifact.load('data/reference/silva-ori/silva-128-99-taxonomy.qza').view(pd.Series)
db_taxa = db_taxa.loc[db_coverage.index]

In [20]:
db_coverage['kingdom'] =  db_taxa.apply(lambda x: x.split(';')[0])
db_coverage['phylum'] = db_taxa.apply(lambda x: x.split(';')[1])
db_coverage['class'] = db_taxa.apply(lambda x:x.split(';')[2])
db_coverage['family'] = db_taxa.apply(lambda x:x.split(';')[4])

In [21]:
class_coverage = db_coverage.groupby('class').sum() / db_coverage.groupby('class').count()
class_coverage.drop(columns=['family'], inplace=True)

In [22]:
db_coverage = db_coverage.loc[db_coverage['kingdom'].isin(['D_0__Archaea', 'D_0__Bacteria'])]

In [23]:
interesting_phyla = ['D_1__Actinobacteria', 
                     'D_1__Bacteroidetes',
                     'D_1__Cyanobacteria',
                     'D_1__Elusimicrobia',
                     'D_1__Euryarchaeota',
                     'D_1__Firmicutes',
                     'D_1__Fusobacteria',
                     'D_1__Lentisphaerae',
                     'D_1__Proteobacteria',
                     'D_1__Spirochaetae',
                     'D_1__Tenericutes',
                     'D_1__Verrucomicrobia'
                     ]
db_coverage = db_coverage.loc[db_coverage['phylum'].isin(interesting_phyla)]

In [24]:
class_match = ['D_2__Actinobacteria',
               'D_2__Alphaproteobacteria',
               'D_2__Chloroplast',
               'D_2__Clostridia',
               'D_2__Coriobacteriia',
               'D_2__Cyanobacteria',
               'D_2__Bacilli',
               'D_2__Bacteroidetes BD2-2',
               'D_2__Bacteroidetes Incertae Sedis',
               'D_2__Bacteroidetes VC2.1 Bac22',
               'D_2__Bacteroidetes vadinHA17',
               'D_2__Bacteroidia',
               'D_2__Betaproteobacteria',
               'D_2__Deltaproteobacteria',
               'D_2__Elusimicrobia',
               'D_2__Epsilonproteobacteria',
               'D_2__Erysipelotrichia',
               'D_2__Gammaproteobacteria',
               'D_2__Methanobacteria',
               'D_2__Mollicutes',
               'D_2__Spirochaetes',
               'D_2__Verrucomicrobiae',
               'D_2__Lentisphaeria',
              ]
db_coverage = db_coverage.loc[db_coverage['class'].isin(class_match)]

In [25]:
silva_class = db_coverage.groupby('class').sum() / db_coverage.groupby('class').count()
silva_class.dropna(axis=1, inplace=True)

In [26]:
silva_class.mean(axis=1).sort_values()

class
D_2__Methanobacteria                 0.340095
D_2__Bacteroidetes Incertae Sedis    0.725230
D_2__Bacilli                         0.734432
D_2__Actinobacteria                  0.742054
D_2__Elusimicrobia                   0.769360
D_2__Spirochaetes                    0.771386
D_2__Bacteroidetes BD2-2             0.777569
D_2__Cyanobacteria                   0.777884
D_2__Bacteroidetes vadinHA17         0.788671
D_2__Mollicutes                      0.790582
D_2__Gammaproteobacteria             0.792754
D_2__Alphaproteobacteria             0.796972
D_2__Betaproteobacteria              0.808499
D_2__Verrucomicrobiae                0.819667
D_2__Deltaproteobacteria             0.820101
D_2__Coriobacteriia                  0.829794
D_2__Epsilonproteobacteria           0.830887
D_2__Bacteroidia                     0.835486
D_2__Chloroplast                     0.837947
D_2__Clostridia                      0.843383
D_2__Lentisphaeria                   0.844569
D_2__Bacteroidetes VC2.1 Bac

In [27]:
silva_class

,v13,v34,v68
class,,,
D_2__Actinobacteria,0.247669,0.986600,0.991894
D_2__Alphaproteobacteria,0.425428,0.988533,0.976955
D_2__Bacilli,0.242487,0.966732,0.994076
D_2__Bacteroidetes BD2-2,0.343985,0.992481,0.996241
D_2__Bacteroidetes Incertae Sedis,0.189990,0.989275,0.996425
D_2__Bacteroidetes VC2.1 Bac22,0.595455,0.995455,0.995455
D_2__Bacteroidetes vadinHA17,0.369281,1.000000,0.996732
D_2__Bacteroidia,0.517037,0.994502,0.994917
D_2__Betaproteobacteria,0.443728,0.985346,0.996424


# Class

## Compare to reference

In [28]:
class_level_n = {
    method: table.copy().norm(axis='sample').collapse(lambda id_, md: md[2], 
                                  norm=False, axis='observation'
                                  ).filter(lambda v, id_, md: v.mean() > 1/1000, axis='observation'
                                          ).to_dataframe().T.sort_index()
    for method, table in table_with_taxa.items()
}


In [29]:
class_long = pd.concat(axis=1, objs=[
    table.reset_index().melt(id_vars='index', var_name='class', value_name=set_).set_index(['index', 'class'])
    for set_, table in class_level_n.items()
])
class_long.index.set_names(['sample-id', 'class'], inplace=True)
class_long_diff = (class_long / class_long[['reference'] * 4].values)
# class_long_diff = (class_long_diff / 
#                    np.vstack([class_long.groupby('class')['reference'].mean().loc[
#                        class_long_diff.index.to_frame()['class']].values] * 4).T)
class_long_diff.dropna(inplace=True)
class_long.reset_index(inplace=True)
class_long = class_long.melt(id_vars=['sample-id', 'class'],
                             var_name='method',
                             value_name='abundance')
class_long_diff.reset_index(inplace=True)
class_long_diff = class_long_diff.melt(id_vars=['sample-id', 'class'],
                             var_name='method',
                             value_name='abundance')

In [30]:
class_long['log-abund'] = np.log10(class_long['abundance'].astype(float).replace({0: np.nan}))
class_long['log-abund'] = class_long['log-abund'].fillna(-5)

In [31]:
len(class_level_n['reference'].columns)

12

In [32]:
tax_order = ['c__Clostridia', 'c__Bacilli', 'c__Erysipelotrichia',
             'c__Bacteroidia',
             'c__Actinobacteria', 'c__Coriobacteriia',
             'c__Gammaproteobacteria', 'c__Betaproteobacteria', 'c__Deltaproteobacteria', 'c__Epsilonproteobacteria',
             'c__Mollicutes',
             'c__Verrucomicrobiae',
             ]

In [33]:
def fit_models(class_, method, class_level=class_level_n):
    ref_ = class_level['reference'][class_]
#     ref_[ref_ <= 1e-7] = 1e-7
#     ref_ = np.log10(ref_)
    other_ = class_level[method][class_]
#     other_[other_ <= 1e-7] = 1e-7
#     other_ = np.log10(other_)
    
    t_stat, t_p = ttest_rel(ref_, other_)
    fit_ = sms.OLS(exog=ref_, endog=other_).fit()
    m = fit_.params[0]
    [[cil, cih]] = fit_.conf_int().values
    
    return pd.Series({'method': method,
                      't_stat': t_stat,
                      't_pval': t_p,
                      'ols_r2': fit_.rsquared,
                      'slope': m,
                      'slope_ci_lo': cil,
                      'slope_ci_hi': cih,
                      'adult_mean': other_.loc[meta['age'] == 'adult'].mean(),
                      'infant_mean': other_.loc[meta['age'] == 'infant'].mean(),
                      }, name=class_)

0.075

In [34]:
class_p_val = pd.concat(axis=1, objs=[
        fit_models(class_,method)
        for class_, method in it.product(class_level_n['reference'].columns, 
                                         methods)
    ]).T
class_p_val['p-adj'] = sms.stats.multipletests(class_p_val['t_pval'], 
                                               method='bonferroni')[1]
class_p_val.index.set_names('class', inplace=True)
class_p_val['slope_off'] = np.absolute(class_p_val['slope'] - 1)
# class_p_val.loc[]
class_p_val.set_index('method', append=True, inplace=True)
# class_p_val = pd.concat(axis=1, objs=[class_p_val, ancom_w])
# class_p_val.dropna(subset=['adult_mean'], inplace=True)


In [35]:
class_p_val.to_csv('data/output/tables/table_s2_class_comparison.tsv', sep='\t')

In [36]:
p_labels = ((np.log10(class_p_val['p-adj'].dropna().astype(float)) * 
             (class_p_val['p-adj'].dropna().astype(float) < 0.05) * 
             (class_p_val.dropna()['slope_off'].astype(float) > 0.05)
             )
           ).apply(
    lambda x: np.absolute(np.ceil(np.max([-3, x])))).astype(int).unstack()
diff_top = class_long_diff.groupby(['class', 'method'])['abundance'].max().unstack()


In [ ]:
fig_ = plt.figure(constrained_layout=True, 
                  dpi=150, 
                  figsize=(6, 5),
                  facecolor='None'
                 )
gs = fig_.add_gridspec(6, 4)
ax = fig_.add_subplot(gs[:, :])
sn.boxplot(x='class', 
             y='abundance', 
             hue='method',
             order=tax_order,
             data=class_long_diff, 
             hue_order=['otus', 'asvs', 'sidle'],
             ax=ax,
           linewidth=1,
#              dodge=True, 
           palette=['w'],
#              palette=['#607381', '#A7B8F8', '#052955'],
             fliersize=0, 
#              edgecolor='None'
            )
ax.plot(ax.get_xlim(), [1, 1], 'k:', linewidth=0.5)
sn.stripplot(x='class', 
             y='abundance', 
             hue='method',
             order=tax_order,
             data=class_long_diff, 
             hue_order=['otus', 'asvs', 'sidle'],
             ax=ax,
             palette=['#AE9C45', '#607381', '#052955'],
             s=1.5, 
             dodge=True,
             edgecolor='None'
            )
ax.xaxis.set_tick_params(rotation=90)
ax.legend_.set_visible(False)
ax.legend(ncol=6)
ax.set_xticklabels([c.split("__")[1] for c in tax_order])
ax.set_ylabel('$\\frac{\\mathrm{Reconstructed}}{\\mathrm{Reference}}$', size=12)


ymin, ymax = ax.get_ylim()
ydelta = ymax - ymin
ax.set_ylim(ymin, ymin + ydelta * 1.05)

for i, class_ in enumerate(tax_order):
    ax.text(i - 0.25,
            diff_top.loc[class_, 'otus'] + ydelta * 0.04, 
            s=''.join(['*'] * (p_labels.loc[class_, 'otus'])),
            ha='center', va='center', size=9,
            )
    ax.text(i,
            diff_top.loc[class_, 'asvs'] + ydelta * 0.05, 
            s=''.join(['*'] * (p_labels.loc[class_, 'asvs'])),
            ha='center', va='center', size=9,
            )
    ax.text(i + 0.25,
            diff_top.loc[class_, 'sidle'] + ydelta * 0.06, 
            s=''.join(['*'] * (p_labels.loc[class_, 'sidle'])),
            ha='center', va='center', size=9,
            )
!mkdir -p data/output/figures/figure2/
fig_.savefig('data/output/figures/figure2/fig2_class_relative_abund.png', facecolor='None', dpi=1200)
fig_.savefig('data/output/figures/figure2/fig2_class_relative_abund.pdf', facecolor='None', dpi=1200)


In [ ]:
outliers = class_p_val.index[(class_p_val['p-adj']  <= 0.1) & 
                             (class_p_val['slope_off'] >= 0.05)].unique()
outliers = class_p_val.loc[outliers].copy()
outliers.drop(columns=['t_stat', 'ols_r2', 't_pval'], inplace=True)
outliers = outliers.astype(float)
outliers.index.to_frame()['class'].unique()

In [ ]:
outlier_classes = class_p_val.groupby('class')['p-adj'].min().index[
    (class_p_val.groupby('class')['p-adj'].min() <= 0.05) & 
    (class_p_val.groupby('class')['slope_off'].max() >= 0.075)
]

In [ ]:
class_p_val.loc[outlier_classes, ['slope', 'slope_ci_lo', 'slope_ci_hi', 'slope_off', 'p-adj']].astype(float).round(3)

In [ ]:
fig_ = plt.figure(constrained_layout=True, 
                  facecolor='None', 
                  figsize=(6, 4),
                  dpi=300)
class_long2 = class_long.set_index(['class', 'method', 'sample-id']).copy()
gs = fig_.add_gridspec(3, 6)
for i, class_ in enumerate(outlier_classes):
    axr = fig_.add_subplot(gs[int(i - np.floor(i / 3) * 3), 3*int(np.floor(i / 3)):3*(int(np.floor(i / 3)) + 1)], 
                           facecolor='None')
    axr.xaxis.set_tick_params(bottom=False, labelbottom=None, top=False, labeltop=False)
    axr.yaxis.set_tick_params(bottom=False, labelbottom=None, top=False, labeltop=False)
    sn.despine(ax=axr, top=True, bottom=True, left=True, right=True)
    axr.set_title(class_.replace('__', '. '))
    if i in {2, 4}:
        axr.set_xlabel("\nReference", size=10)
    if i < 3:
        axr.set_ylabel('Reconstruction\n\n', size=10)
    
# #     ax0 = fig_.add_subplot(gs[i, 0])
# #     sn.stripplot(x='method', 
# #              y='abundance', 
# #              data=class_long.loc[class_long['class'] == class_], 
# #              ax=ax0,
# #              dodge=True, 
# #              palette=['#AE9C45', '#607381', '#A7B8F8', '#052955'],
# #              size=1, 
# #              edgecolor='None'
# #             )
# #     ax0.set_xticklabels(['R', 'O', 'A', 'S'])
   
    ax_w = []
    for j, (method, color) in enumerate(zip(*(['otus', 'asvs', 'sidle'], ['#607381', '#A7B8F8', '#052955']))):
        ax1 = fig_.add_subplot(gs[int(i - np.floor(i / 3) * 3), j + 3*int(np.floor(i / 3))], 
                               sharey=axr, sharex=axr)
        sn.regplot(
            class_long2.loc[(class_, 'reference'), 'abundance'],
            class_long2.loc[(class_, method), 'abundance'],
            scatter=True,
            ci=None,
            line_kws=dict(linewidth=1, alpha=0.5),
            scatter_kws=dict(edgecolor='None', s=2),
            color=color,

        )
        ax1.plot(ax1.get_xlim(), ax1.get_xlim(), 'k:', linewidth=0.5)
        ax1.text(x=ax1.get_xlim()[-1], y=ax1.get_ylim()[0], 
                 s='m={slope:1.2f} '.format(
                     **class_p_val.loc[(class_, method)].astype(float).to_dict()
                     ),
                 size=8,
                 ha='right', va='bottom',
                 )
        ax1.xaxis.set_tick_params(labelsize=8)
        ax1.set_ylabel('')
        ax1.set_xlabel('')
        ax1.yaxis.set_tick_params(labelleft=False, labelsize=8)
        if j == 0:
            ax1.yaxis.set_tick_params(labelleft=True, labelsize=8)
            ax1.set_title('ADBEC'[i], loc='left', size=12)
        
        ax_w.append(ax1)
    for ax, l in zip(*(ax_w, ['OTUs', "ASVs", 'Sidle'])):
        ax.text(x=np.mean(ax1.get_xlim()), 
                y=np.max(ax1.get_ylim()) - np.diff(ax1.get_ylim()) * 0.05, 
                s=l, size=8, va='top', ha='center')
        
fig_.savefig('data/output/figures/figure2/figure_s2_class_slopes.png', dpi=300, facecolor='None')
fig_.savefig('data/output/figures/figure2/figure_s2_class_slopes.pdf', dpi=300, facecolor='None')
